<a href="https://colab.research.google.com/github/xavirubi/tappx-challenge/blob/master/hiring_project/kw_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U keybert

In [ ]:
!pip install -U rake-nltk

In [ ]:
!pip install -U yake

In [ ]:
!pip install -U gensim

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download es_core_news_lg

In [ ]:
import json
# from keybert import KeyBERT
# from rake_nltk import Rake
import nltk
# import yake
from gensim.models import Word2Vec

nltk.download('stopwords')
nltk.download('punkt')

In [23]:
f_articles = open('articles.json')
f_videos = open('videos.json')

articles_data = json.load(f_articles)
videos_data = json.load(f_videos)

article_ids = list(article_id for article_id in articles_data.keys())
video_ids = list(video_id for video_id in videos_data.keys())

In [24]:
def article_text(id):
	return articles_data[id]["text"]

def video_text(id):
	return videos_data[id]["text"]

def article_keywords(id):
	return articles_data[id]["keywords"]

def video_keywords(id):
	return videos_data[id]["keywords"]

In [ ]:
kw_model = KeyBERT()
def keybert_keywords(text):
  return kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3))

In [10]:
rake_model = Rake()
def rake_keywords(text):
  kw_extracted = rake_model.extract_keywords_from_text(text)
  return rake_model.get_ranked_phrases_with_scores()

In [11]:
def yake_keywords(text):
  kw_extractor = yake.KeywordExtractor(lan='en', n=3)
  return kw_extractor.extract_keywords(text)

In [ ]:
for id in article_ids:
  print(f'''TEXT FROM ARTICLE {id}:
  {article_text(id)}
''')
  print(f'''KEYWORDS FROM ARTICLE {id}:
  {article_keywords(id)}
''')
  print(f'''KEYBERT KEYWORDS FROM ARTICLE {id}:
{keybert_keywords(article_text(id))}
''')

  print(f'''RAKE KEYWORDS FROM ARTICLE {id}:
{rake_keywords(article_text(id))}
''')

  print(f'''YAKE KEYWORDS FROM ARTICLE {id}:
{yake_keywords(article_text(id))}



-----------------------------------------------------------------------------------------------------
''')

f_articles.close()

In [ ]:
corpus1 = article_keywords(article_ids[0])
# corpus2 = article_keywords(article_ids[7])
# print(f"CORPUS 1: {corpus1}\n\nCORPUS 2: {corpus2}\n--------------------------------------------------------------\n\n")

model = Word2Vec(corpus1, window=5, min_count=1, workers=4)

model = Word2Vec(corpus1)

In [ ]:
for i in range(0, len(video_ids)):
  corpus2 = video_keywords(video_ids[i])
  similarity_score = model.wv.n_similarity(corpus1, corpus2)
  print(f"SIMILARITY SCORE: {similarity_score}\n")

In [ ]:
import spacy

nlp = spacy.load("es_core_news_lg")
doc1 = nlp(article_text(article_ids[1]))
# doc2 = nlp(video_text(video_ids[0]))
print(article_keywords(article_ids[1]))
# Similarity of two documents
for i in range(0, len(video_ids)):
  doc2 = nlp(video_text(video_ids[i]))
  print(i, doc1.similarity(doc2), video_keywords(video_ids[i]))


In [ ]:
print(article_keywords(article_ids[0]))

def score_func(article_kws, video_kws):
  score = 0
  for article_kw in article_kws:
    for video_kw in video_kws:
      score += nlp(article_kw).similarity(nlp(video_kw))
  return score

for i in range(0, len(video_ids)):
  score = score_func(article_keywords(article_ids[0]), video_keywords(video_ids[i]))
  print(i, score, video_keywords(video_ids[i]), end='\n\n')


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


In [ ]:
candidate_labels = article_keywords(article_ids[0])

for i in range(0, len(video_ids)):
  sequence_to_classify = video_text(video_ids[i])
  print(classifier(sequence_to_classify, candidate_labels, multi_label=True))

#{'labels': ['travel', 'exploration', 'dancing', 'cooking'],
# 'scores': [0.9945111274719238,
#  0.9383890628814697,
#  0.0057061901316046715,
#  0.0018193122232332826],
# 'sequence': 'one day I will see the world'}
